In [ ]:
#Import packages

import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

#Tensorflow
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.layers import (Dense, Dropout,BatchNormalization, Input, Conv1D, Flatten,
                             MaxPooling1D)
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping


#Scikit_learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score, classification_report,
                             ConfusionMatrixDisplay, confusion_matrix)
from sklearn import metrics

In [ ]:
# If using Google Colab.

from google.colab import drive
drive.mount('/content/drive')

##### The ZEMO python package [https://pypi.org/project/ZEMO/] can be used to compute Zernike moments for your images. This library is described in a research paper you can find online [[arxiv:2308.13562](https://arxiv.org/abs/2308.13562v2)].

In [ ]:
zm_nongalaxy = pd.read_csv('/path/to/your/direcotry/zms_data.csv')
zm_galaxy = pd.read_csv('/path/to/your/direcotry/zms_data.csv')

zm_nongalaxy.drop("Unnamed: 0", axis = 1, inplace = True)
zm_galaxy.drop("Unnamed: 0", axis = 1, inplace = True)

zms1 = np.array(zm_nongalaxy)
zmg1 = np.array(zm_galaxy)

all_zm_data = np.concatenate([zmg1,zms1])
len(zmg1), len(zms1), len(all_zm_data)

In [ ]:
# labels due to the length of data 

galaxies_labels = np.zeros(780)
nongalaxy_labels = np.ones(545)
all_labels = np.concatenate([galaxies_labels, nongalaxy_labels])
len(all_labels)

In [ ]:
# Run the model for 10 iterations and calculate the standard deviation.
# You can save the performance materials and the models outputs.

y_test_list=[]
models=[]
Y_pred=[]
accs=[]
cons=[]
test_indx = []

batch_size = 64
NUM_EPOCH = 30

for i in range (10):
    X_train, X_test, y_train, y_test, train_indices, test_indices = train_test_split(all_zm_data, all_labels, np.arange(len(all_labels)), test_size=0.25, shuffle=True, random_state=None)
    y_test_list.append(y_test)
    test_indx.append(test_indices)

    class_weights = {0: len(all_zm_data) / (2*len(zmg1)), 1: len(all_zm_data) / (2*len(zms1))}
    y_train_encoded = to_categorical(y_train, num_classes=2)

    # input value due to the zernike order of 45 (1081 features)
    x = Input(shape=(1081,1))

    #hidden layers
    c0 = Conv1D(256, kernel_size=3, strides=2, padding="same")(x)
    b0 = BatchNormalization()(c0)
    m0 = MaxPooling1D(pool_size=2)(b0)
    d0 = Dropout(0.1)(m0)

    c1 = Conv1D(128, kernel_size=3, strides=2, padding="same")(d0)
    b1 = BatchNormalization()(c1)
    m1 = MaxPooling1D(pool_size=2)(b1)
    d1 = Dropout(0.1)(m1)

    c2 = Conv1D(64, kernel_size=3, strides=2, padding="same")(d1)
    b2 = BatchNormalization()(c2)
    m2 = MaxPooling1D(pool_size=2)(b2)
    d2 = Dropout(0.1)(m2)

    f = Flatten()(d2)

    # output
    de0 = Dense(64, activation='relu')(f)
    de1 = Dense(32, activation='relu')(de0)
    de2 = Dense(2, activation='softmax')(de1)

    model = Model(inputs=x, outputs=de2, name="cnn_zm_45_galaxy_nonegalaxy")
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    optimizer = tf.keras.optimizers.Adam()
    model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

    # Callback Functions
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

    history = model.fit(
    X_train, y_train_encoded,
    batch_size=batch_size,
    epochs=NUM_EPOCH,
    class_weight=class_weights,
    verbose = 1,
    callbacks=es,
    validation_split=0.1
    )
    models.append(history)

    y_pred = model.predict(X_test)
    y_pred_labels = np.argmax(y_pred, axis=1)
    Y_pred.append(y_pred_labels)

    con0 = metrics.confusion_matrix(y_test, y_pred_labels)
    cons.append(con0)

    acc = metrics.accuracy_score(y_test, y_pred_labels)
    accs.append(acc)

Epoch 1/30
14/14 [==============================] - 10s 70ms/step - loss: 0.6499 - accuracy: 0.7088 - val_loss: 4.4980 - val_accuracy: 0.5600
Epoch 2/30
14/14 [==============================] - 0s 21ms/step - loss: 0.4434 - accuracy: 0.8108 - val_loss: 1.9953 - val_accuracy: 0.6800
Epoch 3/30
14/14 [==============================] - 0s 20ms/step - loss: 0.4102 - accuracy: 0.8119 - val_loss: 1.0564 - val_accuracy: 0.7400
Epoch 4/30
14/14 [==============================] - 0s 20ms/step - loss: 0.3713 - accuracy: 0.8331 - val_loss: 0.4429 - val_accuracy: 0.8400
Epoch 5/30
14/14 [==============================] - 0s 20ms/step - loss: 0.3644 - accuracy: 0.8387 - val_loss: 0.4728 - val_accuracy: 0.8300
Epoch 6/30
14/14 [==============================] - 0s 20ms/step - loss: 0.2997 - accuracy: 0.8656 - val_loss: 0.5296 - val_accuracy: 0.8200
Epoch 7/30
14/14 [==============================] - 0s 20ms/step - loss: 0.3276 - accuracy: 0.8656 - val_loss: 0.4056 - val_accuracy: 0.8300
Epoch 8/30
1

In [ ]:
# To save any output you need.

output_el_path = '/path/to/your/direcotry'
import os
pickle_el_filename = 'performances_of_galaxy_nonegalaxy.pickle'
pickle_el_filepath = os.path.join(output_el_path, pickle_el_filename)

with open(pickle_el_filepath, 'wb') as pickle_file:
    pickle.dump(accs, pickle_file)


In [ ]:
print(np.mean(accs), np.std(accs))
plt.plot(accs)